# Empirical IO PhD Class
## Problem Set 0
Jimena, Eyal and Pietro 

September 2020

## Problem 0: Logit Function
** 1. The log-sum-exp function is convex everywhere: **

Pick any distinct $x, y \in \mathbb{R}^{N+1}$ and any $\alpha \in (0,1)$. 
$$f ( \alpha x + ( 1 - \alpha ) y ) = \log \sum_{i=0}^N \exp(\alpha x_i + ( 1 - \alpha ) y_i)$$
Applying Hölder's inequality to $\sum_{i=0}^N \exp(\alpha x_i) \exp( ( 1 - \alpha ) y_i)$ with exponents $\frac{1}{\alpha}$ and $\frac{1}{1-\alpha}$ we get 
$$ \sum_{i=0}^N \exp(\alpha x_i) \exp( ( 1 - \alpha ) y_i ) \leq \left [ \sum_{i=0}^N |\exp(\alpha x_i)|^{\frac{1}{\alpha}}\right ]^\alpha \left [ \sum_{i=0}^N |\exp((1 - \alpha) y_i)|^{\frac{1}{1 - \alpha}} \right ]^{1 - \alpha}$$
Taking logs on both sides and rearranging: 
$$ \log \sum_{i=0}^N exp( \alpha x_i + ( 1 - \alpha ) y_i) \leq \alpha \log \sum_{i=0}^N exp( x_i ) + ( 1 - \alpha )\log \sum_{i=0}^N exp( y_i )  $$
So the function is convex everywhere. 

** 2. Using the max trick: **

Fix some $x \in \mathbb{R}^{N+1}$ and let $m:=\max_i x_i$. Assume wlog $x_0=m$. 
We have 
$$ IV = \log \sum_{i=0}^N exp(x_i) = \log \sum_{i=0}^N exp(x_i) \frac{exp(m)}{exp(m)}  $$
and rearranging 
$$ IV = m + \log ( 1 + \sum_{i=1}^N \exp ( x_i-m ) ) $$
We have rescaled everything relative to the $\max$ and added a constant. With this we take the exponential of smaller numbers and avoid the overflow problem. 

** 3. Comparing it to scipy.misc.logsumexp. Does it appear to suffer from underflow/overflow?
Does it use the max trick?**

First generate a tuple of values which includes some $x_i>600$ and evaluate the function at this point.

In [3]:
import numpy as np
import scipy as sp
from scipy.special import logsumexp

x=np.arange(10, 800, 10)


If we calculate the original IV equation we get the error (the evaluated number is infinity):

In [4]:
IV_1=np.log(np.sum(np.exp(x)))
IV_1

inf

Now we do the *max* trick and compare to the value that we get from the logsumexp function:

In [5]:
m=max(x)
IV=m+np.log(1+np.sum(np.exp(x-m)))
IV


790.6931698812978

In [6]:
logsumexp(x)

790.0000454009604

The logsumexp and the function we modified have similar results but not exactly the same so logsumexp is doing something else to compute IV.

## Problem 1
### Stationary Distribution from eigenvectors
Write a function that computes the ergodic distribution of the matrix
$P$ by examining the properly rescaled eigenvectors and compare your result to $P^{100}$.

We first define the transition matrix P and take the 100th power of it.

In [7]:
P = np.array([[0.2, 0.4, 0.4],[0.1, 0.3, 0.6],[0.5, 0.1, 0.4]])
print("This is the transition matrix P \n", P)
P_100 = np.linalg.matrix_power(P, 100)
print("and this is the 100th power \n", P_100)

import quantecon as qe
stat= qe.markov.core.mc_compute_stationary(P)
print("\n this is the stationary using Quantecon package", stat)

This is the transition matrix P 
 [[0.2 0.4 0.4]
 [0.1 0.3 0.6]
 [0.5 0.1 0.4]]
and this is the 100th power 
 [[0.31034483 0.24137931 0.44827586]
 [0.31034483 0.24137931 0.44827586]
 [0.31034483 0.24137931 0.44827586]]

 this is the stationary using Quantecon package [[0.31034483 0.24137931 0.44827586]]


The iteration approach and the almighty John Stachurski agree on the stationary distribution. Let us try to compute it using the eigenvector method too.
To calculate the stationary distribution we compute the eigenvalues and left eigenvectors of P. One of the eigenvalues is equal to one as P is stochastic.

In [8]:
P_T = np.matrix.transpose(P)
w,v = np.linalg.eig(P_T)
print("the eigenvalues are \n", w)

the eigenvalues are 
 [ 1.  +0.j         -0.05+0.23979158j -0.05-0.23979158j]


The first eigenvalue is equal to one, so the eigenvector associated to it will be our stationary distribution. Notice that the distribution needs to be a row vector so we need to transpose the eigenvectors we found. Indeed the stationary distribution $\Pi$ satisfies $\Pi' = P'\Pi'$. I.e. it is the transpose of the right eigenvector associated with the unit eigenvalue.

In [9]:
eigenvector = np.real( v.transpose()[0])
print("The eigenvector we consider is \n", eigenvector)

The eigenvector we consider is 
 [-0.52048344 -0.40482045 -0.75180941]


Now we need to rescale it so that the elements sum to one.

In [10]:
normalization = np.sum(eigenvector)
Π = eigenvector / normalization 
print("The ergodic distribution found using eigenvectors is \n",Π)

The ergodic distribution found using eigenvectors is 
 [0.31034483 0.24137931 0.44827586]


If we compare with what we found using matrix power, we find the two values are essentially identical.

In [11]:
diff = np.sum ( np.abs(Π -P_100) )
print("The sum of absolute discrepancies between the two matrices is \n",diff)

The sum of absolute discrepancies between the two matrices is 
 1.2684298056342413e-14


### Stationaty Distribution from system of equations
Just to show off our mathematical prowess, this section solves for the stationary distribution in a different way, solving the system of equations that define it.

We want to have $\pi$ such that $ (P^T-I)\pi=0 $ and $\sum_i \pi_i=1$. We can write this as a system of equations $A\pi=b$ with $A^T=[P^T-I, \mathbb{1}]$ and $b=[\mathbb{0}, 1]^T$ so that we can solve for $\pi$ in $A^T A \pi=A^T b$.

We find the same result as with the other methods.

In [12]:
def stat_distr(M):
    A = np.append(np.matrix.transpose(M) - np.identity(len(M)), [np.ones(len(M))], axis=0)
    A_T = np.matrix.transpose(A)
    b = np.matrix.transpose( np.append( [ np.zeros( len(M)) ], [1] ) )
    x = np.linalg.solve( A_T.dot(A), A_T.dot(b) )
    return x

In [13]:
print("the stationary distribution is \n" , stat_distr(P)) 

the stationary distribution is 
 [0.31034483 0.24137931 0.44827586]


## Problem 2: Numerical Integration

We define the function binomial logit using a normal distribution with $\mu=.5$ and $\sigma^2=2$

In [14]:
import scipy as sp
from scipy import stats
def binomiallogit(b, pdf=sp.stats.norm.pdf):
    x=.5
    mu=.5
    sigma=np.sqrt(2)
    return (np.exp(b*x)/(1+np.exp(b*x)))*pdf(b, mu, sigma)
    

    

When we try to integrate over $(-\infty, \infty)$ we get an error, the furthes we can go with the integration limits at around $(-1000, 1000)$ so we take this value as the true value of the integral. The outcome of this computation is called **Quad**

In [15]:
Quad, err=sp.integrate.quad(binomiallogit, -1000, 1000, epsrel=10**(-14))
print('the true value of the integral is', Quad)

the true value of the integral is 0.555939162843465


### Monte Carlo
So we have our baseline and we can now do the other methods and compare to the value that the built in function gives. First we do the Monte Carlo integration. To do so we define a function called **MC** which draws k values of $\beta$ and evaluates the function $\frac{\exp(\beta x)}{1+\exp(\beta x)}$ at the values of $\beta$ that we draw and then we take the average of the value at all such draws.

In [16]:
np.random.seed(7)
def MC(k):
    mu=.5
    sigma=np.sqrt(2)
    B=np.random.normal(mu, sigma, k)
    x=.5
    return np.mean([np.exp(b*x)/(1+np.exp(b*x)) for b in B])



print('The Monte Carlo Approximation using 20 draws is', MC(20))
print('The Monte Carlo Approximation using 400 draws is', MC(400))

The Monte Carlo Approximation using 20 draws is 0.5414657849591988
The Monte Carlo Approximation using 400 draws is 0.5478805176664759


### Gauss-Hermite
The other type of approximation that we do is using the Gauss-Hermite quadrature. We obtain the points and weights from the nuppy function and then take the weighted averge of the modified function. The function is called **GH**

In [17]:
def fun(t):
    x=.5
    return np.exp(t*x)/(1+np.exp(t*x))

def GH(k):
    mu=.5
    sigma=np.sqrt(2)
    pts, weigh= np.polynomial.hermite.hermgauss(k)
    return (1/np.sqrt(np.pi))*np.sum(weigh.dot([fun(np.sqrt(2)*sigma*t+mu) for t in pts]))




print('The Gauss-Hermite approximation with 4 points is', GH(4))
print('The Gauss-Hermite approximation with 12 points is', GH(12))

The Gauss-Hermite approximation with 4 points is 0.5559156754781621
The Gauss-Hermite approximation with 12 points is 0.5559391624283003


Now we make sure that the weights are adding up to one:

In [18]:
pts, weigh= np.polynomial.hermite.hermgauss(4)
np.sum(weigh/np.sqrt(np.pi))==1

True

The weights sum up to one except for some odd values of points that we tried. Probably due to some rounding error. 

### Comparing Approaches
Finally we can compare all the approximations to the "true" value from the built in function. To do so we take the difference of each approximation with Quad (Quad-aporox):

In [19]:
print('the true value is', Quad)
print('The Monte Carlo Approximation with 20 draws gives', MC(20), 'so it is off by', Quad-MC(20))
print('The Monte Carlo Approximation with 400 draws gives', MC(400), 'so it is off by', Quad-MC(400))
print('The Gauss-Hermite Approximation with 4 points gives', GH(4), 'so it is off by', Quad-GH(4))
print('The Gauss-Hermite Approximation with 12 points gives', GH(12), 'so it is off by', Quad-GH(12))

the true value is 0.555939162843465
The Monte Carlo Approximation with 20 draws gives 0.5142823093350095 so it is off by 0.012784531970442825
The Monte Carlo Approximation with 400 draws gives 0.5503042404708859 so it is off by -0.007730789817815609
The Gauss-Hermite Approximation with 4 points gives 0.5559156754781621 so it is off by 2.3487365302887753e-05
The Gauss-Hermite Approximation with 12 points gives 0.5559391624283003 so it is off by 4.151646804118059e-10


We find that the best approximation is given by the Gauss-Hermite with 12 points with an error of the order of $e^{(-10)}$

##  Repeat with two dimensions
First we do the dblquad integration function from python and call it **quad2**


In [20]:

def binomiallogit2(b1, b2, pdf=sp.stats.multivariate_normal.pdf):
    x=np.array([.5, 1])
    mu=np.array([.5, 1])
    sigma=np.array([[np.sqrt(2), 0], [0,1]])
    return (np.exp(b1*x[0]+b2*x[1])/(1+np.exp(b1*x[0]+b2*x[1])))*pdf([b1, b2], mu, sigma)
    

quad2, err2 = sp.integrate.dblquad(binomiallogit2, -300, 300, lambda i: -300, lambda i: 300, epsabs=10**(-14))
print('the baseline value for the two dimension integral is', quad2)

the baseline value for the two dimension integral is 0.7296119616316019


### Monte Carlo 
Now we draw betas from two different distributions. $N(.5, \sqrt{2})$ as before and $N(1, 1)$. Now we have $x=[.5, 1]$. We evaluate the function at these pointa and take the simple average to get the Monte Carlo approximation. We call it **MC2**

In [21]:

def MC2(k):
    x = np.array([.5, 1])
    mu=np.array([.5, 1])
    sigma=np.array([[np.sqrt(2), 0], [0,1]])
    B = np.random.multivariate_normal(mu, sigma, k)
    
    return np.mean([(np.exp(np.inner(t,x))/(1+np.exp(np.inner(t,x)))) for t in B])



print("The sample means for the 20 and 400 Monte Carlo draws in the 2-D case are:")
print("20 draws: ", MC2(20))
print("400 draws: ", MC2(400))


The sample means for the 20 and 400 Monte Carlo draws in the 2-D case are:
20 draws:  0.7137379359868061
400 draws:  0.71184253373983


### Gauss-Hermite 


In [22]:
def fun2(t):
    x = np.array([0.5, 1])
    return np.exp(   np.inner(t,x)   )/(1+np.exp(np.inner(t,x)))

In [23]:
def GH2(k):
    
    mu = np.array([.5, 1])
    sigma = np.array([[np.sqrt(2), 0], [0,1]])
    x = np.array([0.5, 1])
    
    #create grid of points
    
    pts, weight = np.polynomial.hermite.hermgauss(k)
    crd1 = np.tile(pts, k)
    crd2 = np.ndarray.flatten(np.tile(pts, (k,1)), 'F')
    pts2 = np.vstack((crd1,crd2))

    w1 = np.tile(weight, k)
    w2 = np.ndarray.flatten(np.tile(weight, (k,1)), 'F')
    weight2 = np.multiply(w1,w2)
    
    #evaluate summation of function times the weight
    argument = [fun2( np.sqrt(2) * np.matmul(sigma, pts2[:,i]   ) +mu ) for i in range(np.shape(pts2)[1])]
    return (1/np.sqrt(np.pi))* np.sum( weight2.dot(np.array( argument )  ))

In [24]:
GH2(12)

1.2865994085668133

The result that we get from the Gauss-Hermite approximation is pretty bad. We are not sure that we are using the right weights here... 

## Results For 1 Dimension

In [38]:
import pandas as pd
df1=pd.DataFrame(np.array([['True', Quad, 0], ['Monte Carlo 20', MC(20), Quad-MC(20)], ['Monte Carlo 400', MC(400), Quad-MC(400)], ['Gauss-Hermite 4', GH(4), Quad-GH(4)], ['Gauss-Hermite 12', GH(12), Quad-GH(12)]]),
                   columns=['Approach', 'Value', 'Error (True-Approach)'])

df1

,Approach,Value,Error (True-Approach)
0,True,0.555939162843465,0
1,Monte Carlo 20,0.6032955652759326,0.05570675847971651
2,Monte Carlo 400,0.5638568475619502,0.007392160808428305
3,Gauss-Hermite 4,0.5559156754781621,2.3487365302887753e-05
4,Gauss-Hermite 12,0.5559391624283003,4.151646804118059e-10


## Results for 2 Dimensions

In [39]:
df2=pd.DataFrame(np.array([['True', quad2, 0], ['Monte Carlo 20', MC2(20), quad2-MC2(20)], ['Monte Carlo 400', MC2(400), quad2-MC2(400)], ['Gauss-Hermite 4', GH2(4), quad2-GH2(4)], ['Gauss-Hermite 12', GH2(12), quad2-GH2(12)]]),
                   columns=['Approach', 'Value', 'Error (True-Approach)'])

df2

,Approach,Value,Error (True-Approach)
0,True,0.7296119616316019,0
1,Monte Carlo 20,0.7206981612259871,-0.04419511090797479
2,Monte Carlo 400,0.7235145932299892,-0.0059642636537984295
3,Gauss-Hermite 4,1.2864380166055254,-0.5568260549739235
4,Gauss-Hermite 12,1.2865994085668133,-0.5569874469352114


## Integration with a vector of x.

In [29]:
def binomiallogitmixture(X, kk=12):
    
    mu = 0.5
    sigma = np.sqrt(2)
    
    pts, weights = np.polynomial.hermite.hermgauss(kk)
    length_x = np.size(X)

    b = (np.sqrt(2) * sigma * pts) + mu
    frac = np.outer(X,b)
    e = np.exp(frac)
    f = np.divide(e,1+e)
    SIG = f.dot(weights)
    return f.dot(weights)*(1/np.sqrt(np.pi))

In [31]:
binomiallogitmixture([.5,1,10])

array([0.55593916, 0.58994822, 0.56971137])